# Tools with MCP ⏰

The Model Context Protocol (MCP) provides a standardized way to connect AI agents to external tools and data sources. Let's connect to an MCP server using `langchain-mcp-adapters`.

## Setup

Load and/or check for needed environmental variables

In [3]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env(".env")

# Declaring default model
default_model = "google_genai:gemini-2.5-flash"

LANGSMITH_API_KEY=****3d99
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ials
LANGSMITH_ENDPOINT=****.com
GOOGLE_API_KEY=****zZa4
DEFAULT_MODEL=****lash


In [4]:
from langchain_mcp_adapters.client import MultiServerMCPClient
import nest_asyncio

nest_asyncio.apply()

# Connect to the mcp-time server for timezone-aware operations
# This Go-based server provides tools for current time, relative time parsing,
# timezone conversion, duration arithmetic, and time comparison
mcp_client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            "command": "npx",
            "args": ["-y", "@theo.foobar/mcp-time"],
        }
    },
)

# Load tools from the MCP server
mcp_tools = await mcp_client.get_tools()
print(f"Loaded {len(mcp_tools)} MCP tools: {[t.name for t in mcp_tools]}")

Loaded 5 MCP tools: ['add_time', 'compare_time', 'convert_timezone', 'current_time', 'relative_time']


Create an agent with the MCP-provided time tools.

In [5]:
from langchain.agents import create_agent

agent_with_mcp = create_agent(
    model=default_model,
    tools=mcp_tools,
    system_prompt="You are a helpful assistant",
)

Ask about the current time in San Francisco.

In [6]:
result = await agent_with_mcp.ainvoke(
    {"messages": [{"role": "user", "content": "What's the time in SF right now?"}]}
)
for msg in result["messages"]:
    msg.pretty_print()

I0000 00:00:1764447837.668343 3327464 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


================================ Human Message =================================

What's the time in SF right now?
================================== Ai Message ==================================
Tool Calls:
  current_time (e212f749-f10e-46f7-b447-65203c1b0b35)
 Call ID: e212f749-f10e-46f7-b447-65203c1b0b35
  Args:
    timezone: America/Los_Angeles
================================= Tool Message =================================
Name: current_time

2025-11-29T12:23:58-08:00
================================== Ai Message ==================================

The time in San Francisco is 12:23:58 PM PST on November 29, 2025.
